# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
# Read dataframe from csv created in WeatherPy 
city_data = pd.read_csv('output_data/export_city_df.csv')
city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph)
0,SJ,Longyearbyen,15.64,78.22,42.80,75,75,24.16
1,PF,Atuona,-139.03,-9.80,78.49,75,0,19.06
2,SD,Kutum,24.67,14.20,74.77,61,90,11.14
3,GL,Nanortalik,-45.24,60.14,34.36,79,2,16.60
4,US,Kapaa,-159.32,22.08,81.75,66,1,6.93
...,...,...,...,...,...,...,...,...
548,CA,Chapais,-74.85,49.78,51.80,87,90,6.93
549,FR,Arrondissement d'Ajaccio,8.92,42.00,57.20,87,90,6.93
550,CN,Dongsheng,109.98,39.82,61.00,62,66,16.73
551,TD,Moussoro,16.49,13.64,77.34,79,100,19.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Store latitude and longitude in locations
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Store Humidity and convert to float
humidity = city_data["Humidity(%)"].astype(float)
locations

,Latitude,Longitude
0,78.22,15.64
1,-9.80,-139.03
2,14.20,24.67
3,60.14,-45.24
4,22.08,-159.32
...,...,...
548,49.78,-74.85
549,42.00,8.92
550,39.82,109.98
551,13.64,16.49


In [23]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#Filter dataframe to find ideal weather conditions
ideal_city_data = city_data.loc[(city_data["Temperature(F)"] < 80) & 
                                (city_data["Temperature(F)"] > 70) &
                                (city_data["Wind_Speed(mph)"] < 10) &
                                (city_data["Cloudiness(%)"] == 0)]
ideal_city_data = ideal_city_data.reset_index(drop = True)   
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph)
0,SA,Buraidah,43.98,26.33,71.60,37,0,3.36
1,MX,General Bravo,-99.17,25.80,72.14,27,0,5.70
2,AE,Abu Dhabi,54.37,24.47,79.83,57,0,2.24
3,IL,Kiryat Gat,34.76,31.61,73.67,85,0,4.47
4,MX,San Quintín,-115.95,30.48,77.74,53,0,5.70
5,SA,Sakakah,40.21,29.97,75.20,22,0,2.24
6,MG,Ambilobe,49.05,-13.20,71.29,84,0,2.91
7,LY,Waddān,16.14,29.16,71.71,79,0,4.70
8,CN,Fuyang,115.82,32.90,75.04,61,0,7.63
9,BR,Caravelas,-39.25,-17.71,72.81,88,0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# set up additional columns to hold information
ideal_city_data["Hotel Name"] = ""
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph),Hotel Name
0,SA,Buraidah,43.98,26.33,71.60,37,0,3.36,
1,MX,General Bravo,-99.17,25.80,72.14,27,0,5.70,
2,AE,Abu Dhabi,54.37,24.47,79.83,57,0,2.24,
3,IL,Kiryat Gat,34.76,31.61,73.67,85,0,4.47,
4,MX,San Quintín,-115.95,30.48,77.74,53,0,5.70,
5,SA,Sakakah,40.21,29.97,75.20,22,0,2.24,
6,MG,Ambilobe,49.05,-13.20,71.29,84,0,2.91,
7,LY,Waddān,16.14,29.16,71.71,79,0,4.70,
8,CN,Fuyang,115.82,32.90,75.04,61,0,7.63,
9,BR,Caravelas,-39.25,-17.71,72.81,88,0,5.82,


In [30]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_city_data.iterrows():

    # get longtitude and latitude from df
    long = row['Longitude']
    
    lat = row['Latitude']

    # add keyword to params dict
    params['location'] = f"{lat},{long}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        ideal_city_data.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        pass
    

#Drop empty hotel name row/s
ideal_city_data = ideal_city_data[ideal_city_data["Hotel Name"].astype(bool)]
ideal_city_data = ideal_city_data.reset_index(drop = True)
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph),Hotel Name
0,SA,Buraidah,43.98,26.33,71.60,37,0,3.36,الديرة للوحدات السكنية
1,MX,General Bravo,-99.17,25.80,72.14,27,0,5.70,Hotel Don Joaquin
2,AE,Abu Dhabi,54.37,24.47,79.83,57,0,2.24,Marriott Executive Apartments Downtown Abu Dhabi
3,IL,Kiryat Gat,34.76,31.61,73.67,85,0,4.47,שדרות גת 88 קרית גת
4,MX,San Quintín,-115.95,30.48,77.74,53,0,5.70,Hotel Jardines Baja
5,SA,Sakakah,40.21,29.97,75.20,22,0,2.24,ناصر عزام الشايع
6,MG,Ambilobe,49.05,-13.20,71.29,84,0,2.91,Hotel National
7,LY,Waddān,16.14,29.16,71.71,79,0,4.70,محل الشريف محمد العايب
8,CN,Fuyang,115.82,32.90,75.04,61,0,7.63,浙江商贸城南门烟酒店
9,BR,Caravelas,-39.25,-17.71,72.81,88,0,5.82,Pousada Tropical


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_data.iterrows()]
marker_locations = ideal_city_data[["Latitude", "Longitude"]]

In [32]:
# Create a marker_layer using the Hotel Name, City and Country lists to fill the info box
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 4)
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))